In [37]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells...");
                jupyter.actions.call('jupyter-notebook:run-all-cells');
            });
        }
    );
</script>

In [35]:
from IPython.display import display
import math
import piplite
await piplite.install("bqplot")
await piplite.install("ipywidgets")
import bqplot
from bqplot import pyplot as plt
import ipywidgets as widgets

ModuleNotFoundError: No module named 'piplite'

In [ ]:
#Variables

n = 2 #Hill coefficient
Km = 40 #monomeres per cell
beta = 0.2
alpha0 = 0.2164 
alpha = 216.404
#Initial concentrations of proteins/mRNAs
CmtetR0 = 4
Cmlambda0 = 1
CmlacI0 = 1
CptetR0 = 2
Cplambda0 = 1
CplacI0 = 1
CmtetR = 0
Cmlambda = 0
CmlacI = 0
CptetR = 0
Cplambda = 0
CplacI = 0

In [ ]:
#Plot building

Span = 1000
ylacI = [0]*Span
ytetR = [0]*Span
ylambda = [0]*Span
mlacI = [0]*Span
mtetR = [0]*Span
mlambda = [0]*Span
x = [dt*0.1 for dt in range(0, Span)]
dt = 0.1

for i in range(0,Span):
    if (i == 0):
            CmlacI = CmlacI0
            CmtetR = CmtetR0
            Cmlambda = Cmlambda0
            CplacI = CplacI0
            CptetR = CptetR0
            Cplambda = Cplambda0
    else:  
        CmlacI = mlacI[i-1] + alpha0*dt - mlacI[i-1]*dt + alpha*dt/(1+math.pow(ylambda[i-1],n))
        CplacI = ylacI[i-1] - ylacI[i-1]*dt*beta + beta*mlacI[i-1]*dt
        CmtetR = mtetR[i-1] + alpha0*dt - mtetR[i-1]*dt + alpha*dt/(1+math.pow(ylacI[i-1],n))
        CptetR = ytetR[i-1] - ytetR[i-1]*dt*beta + beta*mtetR[i-1]*dt
        Cmlambda = mlambda[i-1] + alpha0*dt - mlambda[i-1]*dt + alpha*dt/(1+math.pow(ytetR[i-1],n))
        Cplambda = ylambda[i-1] - ylambda[i-1]*dt*beta + beta*mlambda[i-1]*dt
    mlacI[i] = CmlacI
    mtetR[i] = CmtetR
    mlambda[i] = Cmlambda
    ylacI[i] = CplacI
    ytetR[i] = CptetR
    ylambda[i] = Cplambda

In [ ]:
fig = plt.figure(title="Repressilator model", legend_location="top-left")

line_chart = plt.plot(x=x, y=[ylacI, ytetR, ylambda],
                     labels=["LacI","TetR", "cI"], colors=['blue', 'green', 'red'],
                     display_legend=True)

plt.xlabel("Time/mRNA half-life time")
plt.ylabel("Protein concentration, in units of Km")

In [ ]:
#Callback
def update_plot(b, a, a0, CmlacI0, CmtetR0, CmcI0, CplacI0, CptetR0, CpcI0):
    ylacI = [0]*Span
    ytetR = [0]*Span
    ylambda = [0]*Span
    mlacI = [0]*Span
    mtetR = [0]*Span
    mlambda = [0]*Span
    for i in range(0,Span):
        if (i == 0):
            CmlacI = CmlacI0
            CmtetR = CmtetR0
            Cmlambda = CmcI0
            CplacI = CplacI0
            CptetR = CptetR0
            Cplambda = CpcI0
        else:  
            CmlacI = mlacI[i-1] + a0*dt - mlacI[i-1]*dt + a*dt/(1+math.pow(ylambda[i-1],n))
            CplacI = ylacI[i-1] - ylacI[i-1]*dt*b + b*mlacI[i-1]*dt
            CmtetR = mtetR[i-1] + a0*dt - mtetR[i-1]*dt + a*dt/(1+math.pow(ylacI[i-1],n))
            CptetR = ytetR[i-1] - ytetR[i-1]*dt*b + b*mtetR[i-1]*dt
            Cmlambda = mlambda[i-1] + a0*dt - mlambda[i-1]*dt + a*dt/(1+math.pow(ytetR[i-1],n))
            Cplambda = ylambda[i-1] - ylambda[i-1]*dt*b + b*mlambda[i-1]*dt
        mlacI[i] = CmlacI
        mtetR[i] = CmtetR
        mlambda[i] = Cmlambda
        ylacI[i] = CplacI
        ytetR[i] = CptetR
        ylambda[i] = Cplambda
    line_chart.y = [ylacI, ytetR, ylambda]

In [ ]:
#widgets
wdj = widgets.interactive(update_plot, b = widgets.FloatText(value=beta, description="beta:", disabled=False),
a = widgets.FloatText(value=alpha, description="alpha:", disabled=False),
a0 = widgets.FloatText(value=alpha0, description="alpha0:", disabled=False),
CmlacI0 = widgets.FloatText(value=CmlacI0, description="CmlacI0:", disabled=False),
CmtetR0 = widgets.FloatText(value=CmtetR0, description="CmtetR0:", disabled=False),
CmcI0 = widgets.FloatText(value=Cmlambda0, description="Cmlambda0:", disabled=False),
CplacI0 = widgets.FloatText(value=CplacI0, description="CplacI0:", disabled=False),
CptetR0 = widgets.FloatText(value=CptetR0, description="CptetR0:", disabled=False),
CpcI0 = widgets.FloatText(value=Cplambda0, description="Cplambda0:", disabled=False))

In [ ]:
display(widgets.HBox([fig, wdj]))